In [11]:
# === Cell 1: Versions & reset ===
import os, random, gc, sys, numpy as np, tensorflow as tf
print("Python:", sys.version)
print("TensorFlow:", tf.__version__)
print("Keras image_data_format:", tf.keras.backend.image_data_format())  # 'channels_last' 여야 정상

# 재현성 + 세션 초기화
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["TF_DETERMINISTIC_OPS"] = "1"
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)
tf.keras.backend.clear_session(); gc.collect()

# GPU 확인
print("Physical GPUs:", tf.config.list_physical_devices('GPU'))


Python: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
TensorFlow: 2.19.0
Keras image_data_format: channels_last
Physical GPUs: []


In [12]:
# === Cell 2: Paths & constants ===
import os

DATA_ROOT = r"C:/Users/moont/Desktop/2025miniinterncode/lumbar_spinal_dataset"
TRAIN_DIR = os.path.join(DATA_ROOT, "training")
TEST_DIR  = os.path.join(DATA_ROOT, "testing")

CLASSES = ["Herniated_Disc", "No_Stenosis", "Thecal_Sac"]
IMG_SIZE = (200, 200)
INPUT_SHAPE = IMG_SIZE + (3,)   # 반드시 3채널
BATCH_SIZE = 32
VAL_SPLIT = 0.2

# EfficientNet 제외. 다음 중 선택:
BACKBONE = "inceptionv3"  # 'resnet50', 'densenet121', 'inceptionv3', 'mobilenetv2'

print("CLASSES:", CLASSES)
print("IMG_SIZE:", IMG_SIZE)
print("INPUT_SHAPE(should be 3ch):", INPUT_SHAPE)
assert INPUT_SHAPE[-1] == 3, "INPUT_SHAPE 마지막 채널이 3이 아닙니다!"


CLASSES: ['Herniated_Disc', 'No_Stenosis', 'Thecal_Sac']
IMG_SIZE: (200, 200)
INPUT_SHAPE(should be 3ch): (200, 200, 3)


In [13]:
# === Cell 3: Datasets (force RGB) ===
train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR, labels="inferred", label_mode="int", class_names=CLASSES,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    validation_split=VAL_SPLIT, subset="training",
    seed=SEED, shuffle=True, color_mode="rgb"
)
val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR, labels="inferred", label_mode="int", class_names=CLASSES,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    validation_split=VAL_SPLIT, subset="validation",
    seed=SEED, shuffle=True, color_mode="rgb"
)
test_ds_raw = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR, labels="inferred", label_mode="int", class_names=CLASSES,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    shuffle=False, color_mode="rgb"
)

print(f"Batches -> Train:{len(train_ds_raw)}  Val:{len(val_ds_raw)}  Test:{len(test_ds_raw)}")

xb, yb = next(iter(train_ds_raw.take(1)))
print("첫 train 배치 shape:", xb.shape)  # (B, 200, 200, 3)
tf.debugging.assert_equal(tf.shape(xb)[-1], 3)


Found 4808 files belonging to 3 classes.
Using 3847 files for training.
Found 4808 files belonging to 3 classes.
Using 961 files for validation.
Found 1158 files belonging to 3 classes.
Batches -> Train:121  Val:31  Test:37
첫 train 배치 shape: (32, 200, 200, 3)


In [14]:
# === Cell 4: Preprocess/Augment (강화판) ===
import tensorflow as tf
from tensorflow.keras import layers

AUTOTUNE = tf.data.AUTOTUNE

# 0) 증강 레이어(훈련 때만)
augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),   # 대비 랜덤
], name="augment")

rescale = layers.Rescaling(1./255, name="rescale")  # 마지막에 사용

# 1) 그레이스케일 → 3채널 유지 (X-ray에 유리)
def to_gray3(imgs):
    g = tf.image.rgb_to_grayscale(imgs)         # [B,H,W,1]
    return tf.tile(g, [1,1,1,3])                 # [B,H,W,3]

# 2) 배치형 미디언 필터 (이미 사용하던 것)
def median_filter_batch(imgs, ksize=3):
    imgs = tf.image.convert_image_dtype(imgs, tf.float32)
    patches = tf.image.extract_patches(
        images=imgs, sizes=[1, ksize, ksize, 1],
        strides=[1,1,1,1], rates=[1,1,1,1], padding='SAME'
    )
    B = tf.shape(imgs)[0]; H = tf.shape(imgs)[1]; W = tf.shape(imgs)[2]; C = tf.shape(imgs)[3]
    patches = tf.reshape(patches, [B, H, W, ksize*ksize, C])
    patches_sorted = tf.sort(patches, axis=3)
    median = tf.gather(patches_sorted, ksize*ksize // 2, axis=3)
    return tf.cast(median, imgs.dtype)

# 3) 언샤프 마스크(로컬 대비/윤곽 강화) - 경계 보존 선호
def gaussian_kernel(ksize=5, sigma=1.0):
    x = tf.range(ksize, dtype=tf.float32) - (ksize-1)/2.0
    gauss = tf.exp(-(x**2)/(2.0*sigma**2))
    gauss = gauss / tf.reduce_sum(gauss)
    kx = gauss[:, None]; ky = gauss[None, :]
    k2d = ky @ kx                                   # [ks, ks]
    k2d = k2d[:, :, None, None]                     # [ks, ks, 1, 1]
    k2d = tf.tile(k2d, [1,1,3,1])                   # RGB 동일 적용
    return k2d

_GBLUR = gaussian_kernel(ksize=5, sigma=1.0)

def unsharp_mask(imgs, amount=0.3):
    # imgs: [B,H,W,3] float32, [0,1] 권장
    blur = tf.nn.depthwise_conv2d(imgs, _GBLUR, strides=[1,1,1,1], padding='SAME')
    sharp = imgs + amount * (imgs - blur)
    return tf.clip_by_value(sharp, 0.0, 1.0)

# 4) per-image 표준화 + 강도 클리핑 (mean±k*std)
def zscore_clip01(imgs, k=2.5, eps=1e-6):
    mean = tf.reduce_mean(imgs, axis=[1,2,3], keepdims=True)
    std  = tf.math.reduce_std(imgs, axis=[1,2,3], keepdims=True)
    z = (imgs - mean) / (std + eps)
    z = tf.clip_by_value(z, -k, k)
    # [-k,k] → [0,1]
    z01 = (z + k) / (2.0*k)
    return z01

# 5) 랜덤 감마 (훈련만)
def random_gamma(imgs, lo=0.9, hi=1.1):
    gamma = tf.random.uniform([tf.shape(imgs)[0], 1, 1, 1], lo, hi)
    # 방어적: 0~1 범위 입력 가정
    imgs = tf.clip_by_value(imgs, 0.0, 1.0)
    return tf.pow(imgs, 1.0 / gamma)

def prepare(ds, training=False):
    if training:
        ds = ds.map(lambda x, y: (augment(x, training=True), y), num_parallel_calls=AUTOTUNE)

    # (선택) X-ray이면 그레이스케일 변환이 대부분 유리
    ds = ds.map(lambda x, y: (to_gray3(x), y), num_parallel_calls=AUTOTUNE)

    # 노이즈 제거
    ds = ds.map(lambda x, y: (median_filter_batch(x, ksize=3), y), num_parallel_calls=AUTOTUNE)

    # 0~1 리스케일 먼저
    ds = ds.map(lambda x, y: (rescale(x), y), num_parallel_calls=AUTOTUNE)

    # 로컬 대비/윤곽 강화
    ds = ds.map(lambda x, y: (unsharp_mask(x, amount=0.3), y), num_parallel_calls=AUTOTUNE)

    # per-image 표준화+클리핑으로 강도 정규화
    ds = ds.map(lambda x, y: (zscore_clip01(x, k=2.5), y), num_parallel_calls=AUTOTUNE)

    # 훈련일 때만 감마 변형(조명/노출 편차 견고화)
    if training:
        ds = ds.map(lambda x, y: (random_gamma(x, 0.9, 1.1), y), num_parallel_calls=AUTOTUNE)

    return ds.cache().prefetch(AUTOTUNE)

# 적용
train_ds = prepare(train_ds_raw, training=True)
val_ds   = prepare(val_ds_raw,   training=False)
test_ds  = prepare(test_ds_raw,  training=False)


xb2, _ = next(iter(train_ds.take(1)))
print("전처리 후 배치 shape:", xb2.shape)  # (B, H, W, 3)
tf.debugging.assert_equal(tf.shape(xb2)[-1], 3)



전처리 후 배치 shape: (32, 200, 200, 3)


In [15]:
# === Cell 5: Backbone builder (no EfficientNet) ===
from tensorflow.keras import applications as KApps

def build_backbone_force_rgb(name: str):
    name = name.lower()
    inp = tf.keras.Input(shape=INPUT_SHAPE, name="force_rgb_input")  # 3채널 고정

    if name == "mobilenetv2":
        base = KApps.MobileNetV2(include_top=False, weights="imagenet", input_tensor=inp)
    elif name == "resnet50":
        base = KApps.ResNet50(include_top=False, weights="imagenet", input_tensor=inp)
    elif name == "densenet121":
        base = KApps.DenseNet121(include_top=False, weights="imagenet", input_tensor=inp)
    elif name == "inceptionv3":
        base = KApps.InceptionV3(include_top=False, weights="imagenet", input_tensor=inp)
    else:
        raise ValueError(f"Unsupported backbone for this notebook: {name}")

    return base


In [16]:
# === Cell 6: backbone build + full model (explicit head, short summary) ===
from tensorflow.keras import layers, models
from collections import deque

def short_summary(model, tail=15):
    """model.summary()의 마지막 tail줄만 출력"""
    buf = []
    model.summary(print_fn=lambda x: buf.append(x))
    for line in deque(buf, maxlen=tail):
        print(line)

def build_deep_model_explicit(backbone_name,
                              input_shape=(200, 200, 3),
                              dropout_rate=0.2):
    # 1) 백본 (이미지넷, 3채널 강제) — Cell 5의 build_backbone_force_rgb 사용
    base = build_backbone_force_rgb(backbone_name)
    base.trainable = False  # Stage 1: 백본 고정

    # 2) 입력/백본 통과
    inputs = base.inputs[0]          # 강제 3채널 input_tensor
    x = base.outputs[0]              # 백본 출력 feature

    # 3) 분류기 헤드 (명시적으로 단계별 구성)
    x = layers.GlobalAveragePooling2D(name="gap")(x)

    # Block 1
    x = layers.Dense(1024, activation="relu", name="dense_1")(x)
    x = layers.BatchNormalization(name="bn_1")(x)
    x = layers.Dropout(dropout_rate, name="dropout_1")(x)

    # Block 2
    x = layers.Dense(512, activation="relu", name="dense_2")(x)
    x = layers.BatchNormalization(name="bn_2")(x)
    x = layers.Dropout(dropout_rate, name="dropout_2")(x)

    # Block 3
    x = layers.Dense(256, activation="relu", name="dense_3")(x)
    x = layers.BatchNormalization(name="bn_3")(x)
    x = layers.Dropout(dropout_rate, name="dropout_3")(x)

    # Block 4
    x = layers.Dense(128, activation="relu", name="dense_4")(x)
    x = layers.BatchNormalization(name="bn_4")(x)
    x = layers.Dropout(dropout_rate, name="dropout_4")(x)

    # Output
    outputs = layers.Dense(len(CLASSES), activation="softmax", name="output")(x)

    model = models.Model(inputs, outputs, name=f"{backbone_name}_deep_model")
    return model, base

# —— 모델 생성 & 짧은 summary 출력 ——
tf.keras.backend.clear_session()
model, base = build_deep_model_explicit(
    backbone_name=BACKBONE,        # 예: "mobilenetv2"
    input_shape=INPUT_SHAPE,
    dropout_rate=0.4
)

print("\n===== ✅ Backbone summary (short) =====")
short_summary(base, tail=15)

print("\n===== ✅ Full model summary (short) =====")
short_summary(model, tail=15)



===== ✅ Backbone summary (short) =====


Model: "inception_v3"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ force_rgb_input     │ (None, 200, 200,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 99, 99,    │        864 │ force_rgb_input[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 99, 99,    │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 99, 99,    │          0

Model: "inceptionv3_deep_model"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ force_rgb_input     │ (None, 200, 200,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 99, 99,    │        864 │ force_rgb_input[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 99, 99,    │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 99, 99,    │ 

In [17]:
# === Cell 8: two-stage training (EarlyStopping 적용) ===
from tensorflow.keras.callbacks import EarlyStopping

def train_two_stage(model, base, train_ds, val_ds,
                    epochs_stage1=30, epochs_stage2=20,
                    finetune_unfreeze_top_k=20,
                    lr_stage1=1e-3, lr_stage2=1e-4,
                    patience=5, monitor="val_loss"):

    early_stop = EarlyStopping(
        monitor=monitor, patience=patience, restore_best_weights=True, verbose=1
    )

    # Stage 1 — 백본 고정
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_stage1),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    print(f"\n[Stage 1] up to {epochs_stage1} epochs (monitor: {monitor}, patience={patience})")
    hist1 = model.fit(
        train_ds, validation_data=val_ds,
        epochs=epochs_stage1, callbacks=[early_stop], verbose=1
    )

    # Stage 2 — 백본 일부 파인튜닝
    for layer in base.layers[-finetune_unfreeze_top_k:]:
        layer.trainable = True

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_stage2),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    print(f"\n[Stage 2] unfreeze top {finetune_unfreeze_top_k} layers, up to {epochs_stage2} epochs")
    hist2 = model.fit(
        train_ds, validation_data=val_ds,
        epochs=epochs_stage2, callbacks=[early_stop], verbose=1
    )
    return hist1, hist2


In [18]:
# === Cell 9: evaluation (정확도/혼동행렬/민감도·특이도/리포트/ROC-AUC) ===
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np

def evaluate_full(model, test_ds, class_names):
    # 예측
    y_true, y_prob = [], []
    for xb, yb in test_ds:
        preds = model.predict(xb, verbose=0)
        y_prob.append(preds)
        y_true.extend(yb.numpy())
    y_true = np.array(y_true)
    y_prob = np.vstack(y_prob)
    y_pred = np.argmax(y_prob, axis=1)

    # 정확도
    acc = accuracy_score(y_true, y_pred)
    print(f"\n✅ 전체 정확도: {acc:.4f}")

    # 혼동행렬
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_names))))
    print("\n[혼동 행렬] (rows=true, cols=pred)\n", cm)

    # 분류 리포트
    print("\n[분류 리포트: Precision / Recall / F1]")
    print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

    # 민감도(Sensitivity), 특이도(Specificity)
    TP = np.diag(cm)
    FN = cm.sum(axis=1) - TP
    FP = cm.sum(axis=0) - TP
    TN = cm.sum() - (TP + FP + FN)
    sensitivity = TP / (TP + FN + 1e-8)
    specificity = TN / (TN + FP + 1e-8)
    print("\n[클래스별 민감도 / 특이도]")
    for i, cls in enumerate(class_names):
        print(f"{cls:15s} | 민감도 {sensitivity[i]:.4f} | 특이도 {specificity[i]:.4f}")

    # ROC-AUC (One-vs-Rest)
    y_true_bin = label_binarize(y_true, classes=list(range(len(class_names))))
    auc_macro = roc_auc_score(y_true_bin, y_prob, multi_class="ovr", average="macro")
    auc_weighted = roc_auc_score(y_true_bin, y_prob, multi_class="ovr", average="weighted")
    print(f"\n[ROC-AUC] macro={auc_macro:.4f} | weighted={auc_weighted:.4f}")


In [19]:
# === Cell 10: train & evaluate (에폭 확대 + 얼리스타핑) ===
_ = train_two_stage(
    model, base,
    train_ds=train_ds, val_ds=val_ds,
    epochs_stage1=30, epochs_stage2=20,     # 반복 횟수 확대
    finetune_unfreeze_top_k=20,             # 상위 20개 레이어 파인튜닝
    lr_stage1=1e-3, lr_stage2=1e-4,
    patience=5, monitor="val_loss"          # 얼리스타핑 기준/인내심
)

evaluate_full(model, test_ds, class_names=CLASSES)



[Stage 1] up to 30 epochs (monitor: val_loss, patience=5)
Epoch 1/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.3574 - loss: 1.5929 - val_accuracy: 0.3590 - val_loss: 1.3454
Epoch 2/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.3678 - loss: 1.3339 - val_accuracy: 0.3996 - val_loss: 1.1096
Epoch 3/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.3754 - loss: 1.2279 - val_accuracy: 0.4402 - val_loss: 1.0602
Epoch 4/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.4029 - loss: 1.1483 - val_accuracy: 0.4443 - val_loss: 1.0610
Epoch 5/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.4130 - loss: 1.1067 - val_accuracy: 0.4339 - val_loss: 1.0558
Epoch 6/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.4227 - loss: 1.0837 - val_accuracy: 0.4620 - val_loss: 1.0446
Epoch 7/30
121/121 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.4440 - loss: 1.0569 - val_accuracy: 0.4537 - val_loss: 1.0561
Epoch 8/30
121/121 ━━━━━━━━━━━━━━━━━━━